In [1]:
seed = 666

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

from time import time

import re, random

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import spacy
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])
import pandas as pd

In [2]:
corpus_df = pd.read_csv('corpus_df.csv').drop(columns=['Unnamed: 0'])
corpus_df.head()
# corpus_df.shape # (1693, 3)

,lyrics,artist,genre
0,I know a girl who thinks it goes she'll make y...,flaming lips,indie
1,"All those bugs buzzin' round your head Well, t...",flaming lips,indie
2,"Something in you, it jitters like a moth And I...",flaming lips,indie
3,Their wasn't any snow on Christmas eve and I k...,flaming lips,indie
4,"You can walk among us, but you can't walk on b...",flaming lips,indie


In [3]:
# corpus_df['genre'].value_counts()
# punk     1097
# indie     596

In [4]:
def lemma_w_sp(df):
    
    """Function that takes in a DF column ['lyrics'];
       returns lemmatized version of that column with spacy"""
    
    df = df.copy()
    
    df['lyrics'] = df['lyrics'].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
    
    return df

In [5]:
corpus_df = lemma_w_sp(corpus_df)
corpus_df.head()

,lyrics,artist,genre
0,I know a girl who think it go -PRON- will make...,flaming lips,indie
1,"All that bug buzzin ' round your head Well , t...",flaming lips,indie
2,"Something in you , it jitter like a moth And I...",flaming lips,indie
3,Their be not any snow on Christmas eve and I k...,flaming lips,indie
4,"You can walk among us , but you can not walk o...",flaming lips,indie


In [6]:
# corpus_df['lyrics_lemma'] = corpus_df["lyrics"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
# corpus_df.head()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(corpus_df['lyrics'], corpus_df['genre'], test_size=.25, random_state=seed,
                                                    stratify=corpus_df['genre'])
# train = pd.concat([X_train, y_train], axis=1)
# test = pd.concat([X_test, y_test], axis=1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1269,), (424,), (1269,), (424,))

In [8]:
# train['genre'].value_counts()
# punk     822
# indie    447

# test['genre'].value_counts()
# punk     275
# indie    149

In [9]:
# def inspect(vectoriser, X):
#     # Fit and transform
#     start = time()
#     print(f"There are {vectoriser.fit_transform(X).shape[1]} columns.\n")
#     end = time()
#     print(f"Took {round((end-start),2)} seconds.\n")
    
#     # Inspect tokens
#     tokens = list(vectoriser.vocabulary_.keys())
#     tokens.sort()
#     print(f"Example tokens: {tokens[:50]}\n")
    
#     # Inspect ignored tokens
#     ignored = vectoriser.stop_words_
#     if len(ignored)==0:
#         print("No token is ignored.")
#     elif len(ignored)>50:
#         print(f"Example ignored tokens: {random.sample(ignored, 50)}")
#     else:
#         print(f"Example ignored tokens: {ignored}")

In [10]:
vectoriser = TfidfVectorizer(token_pattern=r'[a-z]+', stop_words='english', min_df=10, max_df=.97)
#inspect(vectoriser, X_train)

In [11]:
X_train_pr = vectoriser.fit_transform(X_train)
X_train_pr

<1269x978 sparse matrix of type '<class 'numpy.float64'>'
	with 38816 stored elements in Compressed Sparse Row format>

In [12]:
pd.DataFrame(X_train_pr.todense(), columns=vectoriser.get_feature_names())

,accept,act,action,afraid,age,ago,ah,ahead,air,alive,...,y,yea,yeah,year,yes,york,youi,young,youth,yr
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.040428,0.0,...,0.0,0.0,0.0,0.000000,0.274865,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.123202,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1265,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1266,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.126714,0.000000,0.0,0.0,0.0,0.0,0.0
1267,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [24]:
m = RandomForestClassifier(random_state=seed, class_weight='balanced')
m.fit(X_train_pr, y_train)
m.score(X_train_pr, y_train)

1.0

In [25]:
X_test_pr = vectoriser.transform(X_test)
pd.DataFrame(X_test_pr.todense(), columns=vectoriser.get_feature_names())

,accept,act,action,afraid,age,ago,ah,ahead,air,alive,...,y,yea,yeah,year,yes,york,youi,young,youth,yr
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.541642,0.0,0.0,0.000000,...,0.0,0.0,0.074707,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.086564,0.000000,0.0,0.0,0.0,0.0
420,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
421,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
422,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.044154,0.120428,0.062012,0.0,0.0,0.0,0.0


In [26]:
m.score(X_test_pr, y_test)

0.7547169811320755

In [16]:
# new_user_input = 'I am loving this rock n roll song'
# user_input_df = pd.DataFrame([new_user_input], columns=['lyrics'])

In [17]:
# lemmatizer = FunctionTransformer(lemma_w_sp)
# lemmatizer.fit_transform(user_input_df)

In [22]:
text_pipeline = make_pipeline(#FunctionTransformer(lemma_w_sp),
                              TfidfVectorizer(token_pattern=r'[a-z]+', stop_words='english', min_df=10, max_df=.97),
                              RandomForestClassifier(max_depth=5, random_state=seed, class_weight='balanced'))
text_pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.97, min_df=10, stop_words='english',
                                 token_pattern='[a-z]+')),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=5,
                                        random_state=666))])

In [23]:
text_pipeline.predict_proba(['Sometimes I give myself the creeps'])

array([[0.48758333, 0.51241667]])